In [ ]:
"""
tournaments.ipynb

Jupyter Notebook to perform analysis on card market data.

Author: Casey Forey
Date Created: 4/7/24
"""

In [ ]:
%matplotlib inline

# Imports
from importlib import reload
import json 
import os 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import src.constants as c
import src.load.load_set_data as lsd
import src.load.load_tournament_data as ltd
import src.plot.plot_set_data as psd
import src.plot.plot_tournament_data as ptd

In [ ]:
path = 'data/AtomicCards.json' 
f = open(path, encoding= "utf8") 
atm_cards = json.load(f)

path = 'data/AllPrintings.json' 
f = open(path, encoding= "utf8") 
all_cards = json.load(f)

path = 'data/AllPricesToday.json' 
f = open(path) 
prices = json.load(f)

path = 'data/SetList.json' 
f = open(path, encoding= "utf8") 
set_list = json.load(f)

## Modern Data

In [ ]:
base_path: str = os.path.join(c.DATA_DIRECTORY, '2023')
modern_card_counts_df, player_counts_df = ltd.load_format_card_counts(base_path, 'modern')
modern_card_counts_df

In [ ]:
data = atm_cards['data']
cards_set = {}
i = 0
for card in data:
    card_info = data[card][0]
    if 'firstPrinting' in card_info:
        cards_set[card] = card_info['firstPrinting']
    else:
        i +=1
print(i)

In [ ]:
cards_set_df = pd.DataFrame.from_dict(cards_set, orient='index')
cards_set_df.reset_index(inplace= True) 
cards_set_df.rename(columns={'index': 'card_name', 0: 'set_code'},inplace= True) 
cards_set_df

In [ ]:
def add_back_face(df):
    for card in df['card_name']:
        for name in cards_set_df['card_name']:
            front_name = name.split('//')[0].strip()
            if card == front_name and '//' in name:
                df['card_name'].replace(card, name,inplace = True)
    return df

In [ ]:
modern_card_counts_df = add_back_face(modern_card_counts_df)

In [ ]:
# Checking to see if double face cards look correct
for name in modern_card_counts_df['card_name']:
    if "Smelt" in name:
        print(name)

In [ ]:
modern_card_counts_df[modern_card_counts_df['card_name'] == 'Smelt // Herd // Saw']

In [ ]:
modern_card_counts_df = modern_card_counts_df.merge(cards_set_df, on = 'card_name', how = 'left')
modern_card_counts_df

In [ ]:
reload(lsd)

set_release_df: pd.DataFrame = lsd.get_set_and_release_year()
set_release_df

In [ ]:
modern_card_counts_df = modern_card_counts_df.merge(set_release_df, on='set_code', how='left', copy=True)
modern_card_counts_df

In [ ]:
reload(psd)
%matplotlib inline

augmented_modern_card_counts_df: pd.DataFrame = lsd.load_augmented_set_data(all_cards, 'modern')

legend_params: list[tuple] = [
    (2011, 'orange', '--', 'Start of Modern Format (2011)'),
    (2019, 'red', '--', 'Fire Design Principle Implemented (2019)'),
    (2021, 'blue', '--', 'Modern Horizons 2 Released (2021)'),
]
psd.plot_stacked_set_counts(augmented_modern_card_counts_df, set_release_df, 'modern', True, legend_params)

In [ ]:
# Very limited Graph as a proof of concept
modern_card_counts_df.groupby('release_year')['total_count'].sum().plot(kind='bar')

In [ ]:
# reload(ptd)

# ptd.plot_card_counts(modern_card_counts_df,(255/255,122/255,14/255))

### Pioneer Data

In [ ]:
base_path: str = os.path.join(c.DATA_DIRECTORY, '2023')
pioneer_card_counts_df, pioneer_player_counts_df = ltd.load_format_card_counts(base_path, 'pioneer')
pioneer_card_counts_df

In [ ]:
pioneer_card_counts_df = add_back_face(pioneer_card_counts_df)
pioneer_card_counts_df 

In [ ]:
for name in pioneer_card_counts_df['card_name']:
    if "Fable" in name:
        print(name)

In [ ]:
pioneer_card_counts_df = pioneer_card_counts_df.merge(cards_set_df, on = 'card_name', how = 'left')
pioneer_card_counts_df

In [ ]:
pioneer_card_counts_df = pioneer_card_counts_df.merge(set_release_df, on = 'set_code', how = 'left', copy = True)
pioneer_card_counts_df

In [ ]:
# Make set banned cards array and save it as a Numpy array

pioneer_banned_cards: str = """Balustrade Spy
Bloodstained Mire
Expressive Iteration
Felidar Guardian
Field of the Dead
Flooded Strand
Geological Appraiser
Inverter of Truth
Karn, the Great Creator
Kethis, the Hidden Hand
Leyline of Abundance
Lurrus of the Dream-Den
Nexus of Fate
Oko, Thief of Crowns
Once Upon a Time
Polluted Delta
Teferi, Time Raveler
Undercity Informer
Underworld Breach
Uro, Titan of Nature's Wrath
Veil of Summer
Walking Ballista
Wilderness Reclamation
Windswept Heath
Winota, Joiner of Forces
Wooded Foothills
"""
pioneer_banned_cards: np.array = np.array(pioneer_banned_cards.split('\n'))
np.save(os.path.join(c.DATA_DIRECTORY, c.CACHE, 'pioneer_banned_cards'), pioneer_banned_cards)

In [ ]:
# Make set legal sets dataframe and save it as a csv

pioneer_legal_sets: str = """Return to Ravnica
Gatecrash
Dragon's Maze
Magic 2014
Theros
Born of the Gods
Journey into Nyx
Magic 2015
Khans of Tarkir
Fate Reforged
Dragons of Tarkir
Magic Origins
Battle for Zendikar
Oath of the Gatewatch
Welcome Deck 2016
Shadows over Innistrad
Eldritch Moon
Kaladesh
Aether Revolt
Welcome Deck 2017
Amonkhet
Hour of Devastation
Ixalan
Rivals of Ixalan
Dominaria
Core Set 2019
Guilds of Ravnica
Ravnica Allegiance
War of the Spark
Core Set 2020
Throne of Eldraine
Theros Beyond Death
Ikoria: Lair of Behemoths
Core Set 2021
Zendikar Rising
Kaldheim
Strixhaven: School of Mages
Adventures in the Forgotten Realms
Innistrad: Midnight Hunt
Innistrad: Crimson Vow
Kamigawa: Neon Dynasty
Streets of New Capenna
Dominaria United
The Brothers' War
Phyrexia: All Will Be One
March of the Machine
March of the Machine: The Aftermath
Wilds of Eldraine
The Lost Caverns of Ixalan
Murders at Karlov Manor
"""
pioneer_legal_sets: pd.DataFrame = pd.DataFrame({'set_name': pioneer_legal_sets.split('\n')})
pioneer_legal_sets = pioneer_legal_sets.merge(set_release_df, on='set_name')
pioneer_legal_sets.to_csv(os.path.join(c.DATA_DIRECTORY, c.CACHE, 'pioneer_legal_sets.csv'), index=False)
pioneer_legal_sets.head()

In [ ]:
# Make set ban counts dataframe and save it as a csv
pioneer_banned_cards_df: pd.DataFrame = pd.DataFrame({'card_name': pioneer_banned_cards}).merge(cards_set_df, on='card_name')

pioneer_grouped_set_bancount: pd.DataFrame = pioneer_banned_cards_df.groupby(['set_code']).size().reset_index().rename(columns={0: 'num_banned'})
pioneer_final_set_bancount: pd.DataFrame = pioneer_legal_sets.merge(pioneer_grouped_set_bancount, on='set_code', how='left').fillna(0)
pioneer_final_set_bancount.to_csv(os.path.join(c.DATA_DIRECTORY, c.CACHE, 'pioneer_set_ban_counts.csv'), index=False)

In [ ]:
reload(psd)
%matplotlib inline

augmented_pioneer_card_counts_df: pd.DataFrame = lsd.load_augmented_set_data(all_cards, 'pioneer')

legend_params: list[tuple] = [
    (2019, 'red', '--', 'Fire Design Principle Implemented (2019)'),
    (2019, 'blue', ':', 'Pioneer Format Starts (2019)'),
]
psd.plot_stacked_set_counts(augmented_pioneer_card_counts_df, set_release_df, 'pioneer', True, legend_params)

In [ ]:
# reload(ptd)

# ptd.plot_card_counts(pioneer_card_counts_df, (9/255, 121/255, 105/255))

## Legacy

In [ ]:
base_path: str = os.path.join(c.DATA_DIRECTORY, '2023')
legacy_card_counts_df, legacy_player_counts_df = ltd.load_format_card_counts(base_path, 'legacy')
legacy_card_counts_df

In [ ]:
legacy_card_counts_df = add_back_face(legacy_card_counts_df)
legacy_card_counts_df

In [ ]:
for name in legacy_card_counts_df['card_name']:
    if "Fable" in name:
        print(name)

In [ ]:
legacy_card_counts_df = legacy_card_counts_df.merge(cards_set_df, on = 'card_name', how = 'left')
legacy_card_counts_df

In [ ]:
legacy_card_counts_df = legacy_card_counts_df.merge(set_release_df, on = 'set_code', how = 'left', copy = True)
legacy_card_counts_df

In [ ]:
# Make set banned cards array and save it as a Numpy array

legacy_banned_cards: str = """Ancestral Recall
Arcum's Astrolabe
Balance
Bazaar of Baghdad
Black Lotus
Channel
Demonic Consultation
Deathrite Shaman
Demonic Tutor
Dig Through Time
Dreadhorde Arcanist
Earthcraft
Expressive Iteration
Fastbond
Flash
Frantic Search
Gitaxian Probe
Goblin Recruiter
Gush
Hermit Druid
Imperial Seal
Library of Alexandria
Lurrus of the Dream-Den
Mana Crypt
Mana Drain
Mana Vault
Memory Jar
Mental Misstep
Mind Twist
Mishra's Workshop
Mox Emerald
Mox Jet
Mox Pearl
Mox Ruby
Mox Sapphire
Mystical Tutor
Necropotence
Oath of Druids
Oko, Thief of Crowns
Ragavan, Nimble Pilferer
Sensei's Divining Top
Skullclamp
Sol Ring
Strip Mine
Survival of the Fittest
Time Vault
Time Walk
Timetwister
Tinker
Tolarian Academy
Treasure Cruise
Underworld Breach
Vampiric Tutor
Wheel of Fortune
White Plume Adventurer
Windfall
Wrenn and Six
Yawgmoth's Bargain
Yawgmoth's Will
Zirda, the Dawnwaker
"""
legacy_banned_cards: np.array = np.array(legacy_banned_cards.split('\n'))
np.save(os.path.join(c.DATA_DIRECTORY, c.CACHE, 'legacy_banned_cards'), legacy_banned_cards)

# Make set legal sets dataframe and save it as a csv

legacy_legal_sets: str = [set_entry['name'] for set_entry in set_list['data']]
legacy_legal_sets: pd.DataFrame = pd.DataFrame({'set_name': legacy_legal_sets})
legacy_legal_sets = legacy_legal_sets.merge(set_release_df, on='set_name')
legacy_legal_sets.to_csv(os.path.join(c.DATA_DIRECTORY, c.CACHE, 'legacy_legal_sets.csv'), index=False)
legacy_legal_sets.head()

# Make set ban counts dataframe and save it as a csv
legacy_banned_cards_df: pd.DataFrame = pd.DataFrame({'card_name': legacy_banned_cards}).merge(cards_set_df, on='card_name')

legacy_grouped_set_bancount: pd.DataFrame = legacy_banned_cards_df.groupby(['set_code']).size().reset_index().rename(columns={0: 'num_banned'})
legacy_final_set_bancount: pd.DataFrame = legacy_legal_sets.merge(legacy_grouped_set_bancount, on='set_code', how='left').fillna(0)
legacy_final_set_bancount.to_csv(os.path.join(c.DATA_DIRECTORY, c.CACHE, 'legacy_set_ban_counts.csv'), index=False)


In [ ]:
reload(psd)
%matplotlib inline

augmented_legacy_card_counts_df: pd.DataFrame = lsd.load_augmented_set_data(all_cards, 'legacy')

legend_params: list[tuple] = [
    (1997, 'blue', ':', 'Legacy Format Starts (1997)'),
    (2019, 'red', '--', 'Fire Design Principle Implemented (2019)'),
]
psd.plot_stacked_set_counts(augmented_legacy_card_counts_df, set_release_df, 'legacy', True, legend_params)

In [ ]:
# reload(ptd)

# ptd.plot_card_counts(legacy_card_counts_df, (255/255, 215/255, 0/255))